# A SIMPLIFIED MLOPS GUIDE USING MLFLOW

MLOps is one of the crucial phase in a machine learning lifecycle to keep machine learning models maintained and perform well. There's an open source MLOps tool that easily to use, comprehensive ML library, and manage end-to-end ML workkflows from development to production. It is called [MLflow](https://mlflow.org). Here's how to understand by doing a fundamental implementation using ML flow. Let's just dive into the notebook.

[SOURCE](https://mlflow.org/docs/latest/getting-started/logging-first-model/index.html)

# 1. Install MLflow (Including libraries and dependencies)

run it on your IDE terminal

`pip install mlflow`

# 2. Launch the Mlflow Tracking Server

Run it on your IDE terminal

`mlflow server --host 127.0.0.1 --port 8080`

You need to launch the MLflow tracking server and always keep it ongoing during the tutorial, if you close/kill the terminal it'll shut down the server. The screen should be showing like [this](../MLOps/Screenshots/ss1.png) (or ss1.png in the Screenshots folder)

# 3. Using the MLFflow Client API using `MLflowClient`
it could use for :
1. Initiate a new Experiment.
2. Start Runs within an Experiment.
3. Document parameters, metrics, and tags for your Runs.
4. Log artifacts linked to runs, such as models, tables, plots, and more.



## 3.1 Import Dependencies

In [1]:
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestClassifier

#### Configuring the MLflow Tracking Client

In [2]:
client = MlflowClient(tracking_uri="http://127.0.0.1:8080")

# Default Experiment
It's an outset of starting MLflow Tracking Server that if you don’t explicitly create a new experiment in MLflow, any run data is automatically stored in the “Default Experiment” so that it isn’t lost.

## 3.2 Searching Experiments

[mlflow.client.MlflowClient.search_experiments()](https://mlflow.org/docs/latest/python_api/mlflow.client.html#mlflow.client.MlflowClient.search_experiments)

In [3]:
all_experiments = client.search_experiments()
print(all_experiments)

# the output would be a list of Experiment objects

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1736852050555, experiment_id='0', last_update_time=1736852050555, lifecycle_stage='active', name='Default', tags={}>]


To get familiar with accessing elements from returned collections from MLflow APIs, extract the `name` and the `lifecycle_stage` from the `search_experiments()` query and extract these attributes into a dict.

In [4]:
default_experiment = [
    {
        "name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage
    }
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}


# 4. Creating experiments

## Viewing the MLFlow UI
you could see the default experiment with no run data at [http://127.0.0.1:8080](http://127.0.0.1:8080)

## Notes on Tags vs Experiments
While MLflow does provide a default experiment, it primarily serves as a ‘catch-all’ safety net for runs initiated without a specified active experiment. However, it’s not recommended for regular use. Instead, creating unique experiments for specific collections of runs offers numerous advantages, as we’ll explore below.

**Benefits of Defining Unique Experiments**:

1. **Enhanced Organization**: Experiments allow you to group related runs, making it easier to track and compare them. This is especially helpful when managing numerous runs, as in large-scale projects.
2. **Metadata Annotation**: Experiments can carry metadata that aids in organizing and associating runs with larger projects.

Consider the scenario below: we’re simulating participation in a large demand forecasting project. This project involves building forecasting models for various departments in a chain of grocery stores, each housing numerous products. Our focus here is the ‘produce’ department, which has several distinct items, each requiring its own forecast model. Organizing these models becomes paramount to ensure easy navigation and comparison.

**When Should You Define an Experiment?**

The guiding principle for creating an experiment is the consistency of the input data. If multiple runs use the same input dataset (even if they utilize different portions of it), they logically belong to the same experiment. For other hierarchical categorizations, using tags is advisable.

**[NOTE](https://mlflow.org/docs/latest/getting-started/logging-first-model/step3-create-experiment.html)**

While the business product hierarchy in this case doesn’t explicitly need to be captured within the tags, there is nothing preventing you from doing so. There isn’t a limit to the number of tags that you can apply. Provided that the keys being used are consistent across experiments and runs to permit search to function properly, any number of arbitrary mappings between tracked models and your specific business rules can be applied.


## 4.1 Create the Apples Experiment with Meaningfull Tags

In [5]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "This is the grocery forecasting project. "
    "This experiment contains the produce models for apples."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "grocery-forecasting",
    "store_dept": "produce",
    "team": "stores-ml",
    "project_quarter": "Q3-2023",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
produce_apples_experiment = client.create_experiment(
    name="Apple_Models1", tags=experiment_tags
)


# 5. Searching Experiments


## 5.1 Search based on tags

In [12]:
# Use search_experiments() to search on the project_name tag key

apples_experiment = client.search_experiments(
    filter_string="tags.`project_name` = 'grocery-forecasting'"
)

print((apples_experiment[0])) # this output is an Experiment object not a dict
# print(vars(apples_experiment[0])) # this output is a dict o
# the results are the metadata from the experiment that you created

<Experiment: artifact_location='mlflow-artifacts:/209577529335836473', creation_time=1736946368182, experiment_id='209577529335836473', last_update_time=1736946368182, lifecycle_stage='active', name='Apple_Models1', tags={'mlflow.note.content': 'This is the grocery forecasting project. This '
                        'experiment contains the produce models for apples.',
 'project_name': 'grocery-forecasting',
 'project_quarter': 'Q3-2023',
 'store_dept': 'produce',
 'team': 'stores-ml'}>


# 6. Create a Dataset about apples

## 6.1 Defining a dataset generator


In [14]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


def generate_apple_sales_data_with_promo_adjustment(
        base_demand: int = 1000, n_rows: int = 5000
):
    """
    Generates a synthetic dataset for predicting apple sales demand with seasonality
    and inflation.

    This function creates a pandas DataFrame with features relevant to apple sales.
    The features include date, average_temperature, rainfall, weekend flag, holiday flag,
    promotional flag, price_per_kg, and the previous day's demand. The target variable,
    'demand', is generated based on a combination of these features with some added noise.

    Args:
        base_demand (int, optional): Base demand for apples. Defaults to 1000.
        n_rows (int, optional): Number of rows (days) of data to generate. Defaults to 5000.

    Returns:
        pd.DataFrame: DataFrame with features and target variable for apple sales prediction.

    Example:
        >>> df = generate_apple_sales_data_with_seasonality(base_demand=1200, n_rows=6000)
        >>> df.head()
    """

    # Set seed for reproducibility
    np.random.seed(9999)

    # Create date range
    dates = [datetime.now() - timedelta(days=i) for i in range(n_rows)]
    dates.reverse()

    # Generate features
    df = pd.DataFrame(
        {
            "date": dates,
            "average_temperature": np.random.uniform(10, 35, n_rows),
            "rainfall": np.random.exponential(5, n_rows),
            "weekend": [(date.weekday() >= 5) * 1 for date in dates],
            "holiday": np.random.choice([0, 1], n_rows, p=[0.97, 0.03]),
            "price_per_kg": np.random.uniform(0.5, 3, n_rows),
            "month": [date.month for date in dates],
        }
    )

    # Introduce inflation over time (years)
    df["inflation_multiplier"] = (
            1 + (df["date"].dt.year - df["date"].dt.year.min()) * 0.03
    )

    # Incorporate seasonality due to apple harvests
    df["harvest_effect"] = np.sin(2 * np.pi * (df["month"] - 3) / 12) + np.sin(
        2 * np.pi * (df["month"] - 9) / 12
    )

    # Modify the price_per_kg based on harvest effect
    df["price_per_kg"] = df["price_per_kg"] - df["harvest_effect"] * 0.5

    # Adjust promo periods to coincide with periods lagging peak harvest by 1 month
    peak_months = [4, 10]  # months following the peak availability
    df["promo"] = np.where(
        df["month"].isin(peak_months),
        1,
        np.random.choice([0, 1], n_rows, p=[0.85, 0.15]),
    )

    # Generate target variable based on features
    base_price_effect = -df["price_per_kg"] * 50
    seasonality_effect = df["harvest_effect"] * 50
    promo_effect = df["promo"] * 200

    df["demand"] = (
                           base_demand
                           + base_price_effect
                           + seasonality_effect
                           + promo_effect
                           + df["weekend"] * 300
                           + np.random.normal(0, 50, n_rows)
                   ) * df[
                       "inflation_multiplier"
                   ]  # adding random noise

    # Add previous day's demand
    df["previous_days_demand"] = df["demand"].shift(1)
    df["previous_days_demand"].fillna(
        method="bfill", inplace=True
    )  # fill the first row

    # Drop temporary columns
    df.drop(columns=["inflation_multiplier", "harvest_effect", "month"], inplace=True)

    return df


In [15]:
data = generate_apple_sales_data_with_promo_adjustment(base_demand=1_000, n_rows=1_000)

data[-20:]

/tmp/ipykernel_7846/3829094379.py:89: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["previous_days_demand"].fillna(
/tmp/ipykernel_7846/3829094379.py:89: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["previous_days_demand"].fillna(


,date,average_temperature,rainfall,weekend,holiday,price_per_kg,promo,demand,previous_days_demand
980,2024-12-27 22:38:02.645092,34.130183,1.454065,0,0,1.449177,0,971.802447,1001.085782
981,2024-12-28 22:38:02.645090,32.353643,9.462859,1,0,2.856503,0,1136.951553,971.802447
982,2024-12-29 22:38:02.645088,18.816833,0.391470,1,0,1.326429,0,1281.352029,1136.951553
983,2024-12-30 22:38:02.645084,34.533012,2.120477,0,0,0.970131,0,1039.385504,1281.352029
984,2024-12-31 22:38:02.645082,23.057202,2.365705,0,0,1.049931,0,991.427049,1039.385504
985,2025-01-01 22:38:02.645080,34.810165,3.089005,0,0,2.035149,0,1002.564672,991.427049
986,2025-01-02 22:38:02.645078,29.208905,3.673292,0,0,2.518098,0,1086.143402,1002.564672
987,2025-01-03 22:38:02.645076,16.428676,4.077782,0,0,1.268979,0,1093.207186,1086.143402
988,2025-01-04 22:38:02.645075,32.067512,2.734454,1,0,0.762317,0,1396.939894,1093.207186
989,2025-01-05 22:38:02.645073,31.938203,13.883486,1,0,1.153301,0,1321.409540,1396.939894


# 7. Logging our first runs with MLflow

Core features of MLflow Tracking:
- Making use of the start_run context for creating and efficiently managing runs.
- An introduction to logging, covering tags, parameters, and metrics.
- Understanding the role and formation of a model signature.
- Logging a trained model, solidifying its presence in our MLflow run.
